In [1]:
import sys
sys.path.append("../")
import concurrent.futures
from tqdm import tqdm
import pandas as pd
import numpy as np
import openai
from io import StringIO
import re
import time
import math
import warnings
warnings.filterwarnings('ignore')

In [2]:
product_name = ["NIKI", "LE 5 A 7", "JAMIE", "ENVELOPE", "KATE", "MANHATTAN", "KATE TASSEL", "ICARE", "PUFFER", "SUNSET", "SOLFERINO", "LE 37", "LOULOU", "CASSANDRA", "LE 57", "KAIA", "CALYPSO", "SHOPPING SAINT LAURENT", "CASSANDRE MATELASSE", "NOLITA", "GABY", "SADE", "外套", "裙子", "T恤", "其他成衣", "男士成衣", "女士成衣", "高跟鞋", "其他鞋履", "皮带", "墨镜", "饰品"]
content_category = ["UGC", "PGC", "销售人员", "明星代言", "工艺与传承", "OOTD", "旅行体验", "女性力量", "好物推荐", "包包测评", "购物分享"]

In [ ]:
system_prompt = f"""
    你是一个为 YSL 时尚社交聆听系统 服务的打标助手。你的任务是处理多条社交媒体帖子，并为每条帖子提取结构化标签。

⸻

以 CSV 格式 返回结果，例如：

    id,product_name,material,shape,bag_size,color,visual_element,functionality,style,emotion_expression,wearing_occation,content_category
	"5","Niki","羊皮革","方包","小号","黑色","金色经典logo|品牌徽标磁扣开合","斜挎|单肩包","经典","太爱了|推荐","通勤|拍照|日常","种草分享"

	以上只是一个例子，在实际标注过程中，如果信息缺失，请不要将其作为正确的信息使用。

    每条帖子由 id 标识。对于每条帖子，严格按照以下顺序提取 12 个字段：
	•	id：原始帖子 ID（原样复制）。
	•	product_name：提到的产品是什么，只应识别 YSL 时尚产品。一些名称示例如下：["{'","'.join(product_name)}”]。
	•	material：产品的主要材质（例如：油蜡皮、麂皮、羊皮、牛皮、鳄鱼皮、鳄鱼压花、蛇皮、珍稀皮、蜥蜴皮、鸵鸟皮等）。
	•	shape：与产品造型相关的词语（例如：托特包、月牙包、手拿包、盒子包、单肩包、双肩包、饺子包、斜挎包、水桶包、旅行包、卡包、钱包、猫眼镜、直筒连衣裙、修身上衣、喇叭裤等）。你可以结合对 YSL 产品的常识理解。
	•	bag_size：提及的包的尺寸（例如：迷你、小号、中号、24x18x6cm 等）。
	•	color：提及的产品颜色（例如：棕色、黑色、粉色等）。
	•	visual_element：标志性元素或印花（例如：豹纹、格纹、老花、YSL logo、印花等）。
	•	functionality：提及的突出产品功能（例如：斜挎包、单肩包、旅行包、大容量、可拆卸肩带、多种背法、有隔层、自重轻等）。
	•	style：美学风格（例如：优雅风、复古、cleanfit、大包回潮、Y2K、高级等）。同时考虑显式提及和隐含含义。例如：“草编”暗示“度假风”，“A4”暗示“大包回潮”，“无 logo”暗示“cleanfit”。
	•	emotion_expression：与提及产品的主观态度相关的词语（例如：气场全开、便宜、性价比等）。
	•	wearing_occation：使用场景（例如：通勤、度假、逛街、夏日、节日等）。同时考虑显式提及和隐含含义。例如：“可放电脑”暗示“通勤”，“打卡”暗示“拍照”。
	•	content_category：根据帖子内容推断，仅可从 ["{'","'.join(content_category)}”] 中选择。选择 2–3 个类别，当证据较弱时可以进行合理猜测。

⸻

重要规则：
	•	除了表头外，每条帖子仅对应一行。如果发现多款 YSL 产品，请将它们放在同一行。确保每行包含 12 个字段。
	•	如果发现多个关键词，用 | 分隔。不要在关键词中包含英文逗号或中文逗号，如有必要请将其拆分为多个关键词。
	•	只考虑与 YSL 时尚产品 相关的内容。忽略其他品牌和其他产品类别。
	•	帖子是中文的，如果关键词中出现表情符号，请将其转化为可理解的文字。例如：🉑、🈚️ 等。
	•	保持每个关键词简短且直接。尽可能多地提取关键词。
	•	对于缺失或不可用的值，用英文引号括起的 "no" 表示——不要留空。
	•	如果某条帖子中未发现任何 YSL 时尚产品，请跳过（不输出该行）。
	•	不要解释或总结——只输出 CSV 行。

    """

In [4]:
def is_strict_bad(df):
    # Expected columns
    expected_cols = [
        "id", "product_name", "material", "shape", "bag_size", "color", "visual_element", "functionality",
        "style", "emotion_expression", "wearing_occation", "content_category"]
    if list(df.columns) != expected_cols:
        return True

    # 1. Retry if empty table
    if df.empty:
        return True
    # 2. id must be all valid integers
    try:
        # This ensures no non-integer, NaN, or float
        if not df["id"].apply(lambda x: str(x).isdigit()).all():
            return True
    except Exception:
        return True

    return False

In [5]:
# **switch ds version here

import time
from dashscope import Generation

# -------- 你的 API Key 直接写这里 --------
DASHSCOPE_API_KEY = "sk-c2bc15edd94a40d883cfb5f7362b6156"   # ⚠️ 注意替换为你自己的 key
# --------------------------------------

def Ask_GenAI(prompt, model="deepseek-r1-0528", enable_thinking=False):
    """
    使用阿里云百炼 DashScope API 调用 DeepSeek 模型。
    输入 prompt，返回字符串输出内容。
    """
    # 组装 messages，与 OpenAI 格式相同
    messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": prompt}
              ]
    
    # 调用 dashscope Generation 接口
    response = Generation.call(
        api_key=DASHSCOPE_API_KEY,
        model=model,
        messages=messages,
        result_format="message",      # 返回标准 message 格式
        enable_thinking=enable_thinking,  # 可选：是否启用思考模式
        stream=False,                 # 非流式，一次性拿完整输出
    )

    # 提取回复内容
    if hasattr(response, "output") and response.output and response.output.choices:
        content = response.output.choices[0].message.content
    else:
        content = "(No response)"
    
    time.sleep(1)  # 控制调用速率
    return content

In [6]:
# Format batched content for GPT
def make_user_message(df_batch):
    rows = []
    for _, row in df_batch.iterrows():
        id = row["index"]
        content = row["title_content_hashtag_cleaned"]
        rows.append(f"id: {id}\ncontent: {content}\n")
    return "Tag the following posts one by one: \n\n" + "\n End of the post \n".join(rows)

def contains_invalid_keywords(text, invalid_list):
    if pd.isna(text):
        return False
    for kw in invalid_list:
        if kw in str(text).split("|"):
            return True
    return False

In [7]:
# **switch the cleaned df ver here

cleaned_df = pd.read_csv("/root/ysl_project_clean/Output/cleaned_df_v2.csv").iloc[:1000,]

# 确保存在名为 "index" 的列（make_user_message 需要）
cleaned_df_content = cleaned_df.reset_index(drop=False)            # 生成一列 'index'

# 只保留 make_user_message 用到的两列
cleaned_df_content = cleaned_df_content[['index', 'title_content_hashtag_cleaned']]

cleaned_df_content.head(5)

,index,title_content_hashtag_cleaned
0,0,来了｜秋天得必备麂皮𝐈𝐂𝐀𝐑𝐈𝐍𝐎。内容：Y家的麂皮棕配上大衣和毛衣真的高级松弛感拉满～ 包...
1,1,YSL·潮流百搭款🕶️。内容：YSL板材微翘猫眼款🕶️ 板材前框加上金属镜腿，ysl交织lo...
2,2,BELLA HADID IN圣罗兰。内容：#ysl #超模 #骨子里的优雅 #时尚女魔头 #...
3,3,听说SKP所有品类打骨折？。内容：ysl全品类参加活动， 力度开店以来最大一次 抓住机会哦 ...
4,4,圣罗兰墨镜玳瑁色真好看。内容：#昆明恒隆圣罗兰 #设计感墨镜 #时尚太阳眼镜 #时尚潮流眼镜...


In [8]:
Ask_GenAI('HI')

'id,product_name,material,shape,bag_size,color,visual_element,functionality,style,emotion_expression,wearing_occation,content_category'

In [9]:
from io import StringIO
import pandas as pd
import re
import time
import concurrent.futures
from tqdm import tqdm

# ====== 可调参数 ======
N_SAMPLES   = 1000        # 取样数量，正式运行改为 len(df)
BATCH_SIZE  = 8          # 每批多少行
MAX_WORKERS = 10         # 并行线程数（依接口限流调节）
RETRY_LIMIT = 1          # 每个批次失败后最多再重试几次
SLEEP_SEC   = 0.5        # 每批次之间的停顿（防止请求过密）

# ====== 取数据 ======
source_df = cleaned_df_content.iloc[:N_SAMPLES].copy()

expected_cols = [
    "id", "product_name", "material", "shape", "bag_size", "color",
    "visual_element", "functionality", "style", "emotion_expression",
    "wearing_occation", "content_category"
]

results = []
failed_logs = []

print(f"开始并行处理数据... 共 {len(source_df)} 条，批大小 {BATCH_SIZE}，线程数 {MAX_WORKERS}\n")

# ====== 定义处理函数 ======
def process_batch(start, batch_df):
    """处理单个批次，含重试机制"""
    for attempt in range(RETRY_LIMIT + 1):
        try:
            # 构造 prompt
            prompt = make_user_message(batch_df)

            # 调用模型
            csv_output = Ask_GenAI(prompt)
            csv_output = re.sub(r"^```csv\s*|\s*```$", "", csv_output.strip(), flags=re.MULTILINE)

            # 解析 CSV
            df_result = pd.read_csv(StringIO(csv_output))

            # 列对齐
            for c in expected_cols:
                if c not in df_result.columns:
                    df_result[c] = pd.NA
            df_result = df_result[expected_cols]

            return df_result

        except Exception as e:
            failed_logs.append({
                "batch_range": f"{start}-{start + len(batch_df) - 1}",
                "attempt": attempt + 1,
                "reason": str(e)
            })
            time.sleep(0.3)

    # 所有尝试都失败
    return None

# ====== 构造批次列表 ======
batches = [(i, source_df.iloc[i:i + BATCH_SIZE]) for i in range(0, len(source_df), BATCH_SIZE)]

# ====== 并行执行 ======
with concurrent.futures.ThreadPoolExecutor(max_workers=MAX_WORKERS) as executor:
    futures = {executor.submit(process_batch, i, df): i for i, df in batches}

    for future in tqdm(concurrent.futures.as_completed(futures), total=len(futures), desc="Processing batches"):
        result = future.result()
        if result is not None:
            results.append(result)
        time.sleep(SLEEP_SEC)

# ====== 汇总结果 ======
print("\n✅ 处理完成。成功批次数：", len(results), " 失败批次数：", len(failed_logs))
if failed_logs:
    print("⚠️ 失败批次示例：", failed_logs[:3])

if results:
    content_tagging = pd.concat(results, ignore_index=True)
    content_tagging = content_tagging[expected_cols]
    print(f"📁 已生成打标结果 content_tagging，形状: {content_tagging.shape}")
else:
    print("❌ 无有效结果，未生成文件。")

开始并行处理数据... 共 1000 条，批大小 8，线程数 10



Processing batches:   0%|          | 0/125 [00:00<?, ?it/s]

Processing batches: 100%|██████████| 125/125 [26:53<00:00, 12.91s/it] 


✅ 处理完成。成功批次数： 125  失败批次数： 0
📁 已生成打标结果 content_tagging，形状: (729, 12)


In [10]:
import pandas as pd

# ✅ 将 index 变为普通列
cleaned_df_content = cleaned_df_content.reset_index()

# ✅ 按 index = id 合并
content_tagging_with_content = pd.merge(
    cleaned_df_content,
    content_tagging,
    how="inner",          # inner = 只保留匹配上的；可改为 "left" 保留全部 content
    left_on="index",
    right_on="id"
)

# ✅ 删除重复列（可选）
content_tagging_with_content = content_tagging_with_content.drop(columns=["index"])

print(f"✅ 合并完成，共 {content_tagging_with_content.shape[0]} 行，{content_tagging_with_content.shape[1]} 列。")
display(content_tagging_with_content.sample(5))

✅ 合并完成，共 711 行，14 列。


,level_0,title_content_hashtag_cleaned,id,product_name,material,shape,bag_size,color,visual_element,functionality,style,emotion_expression,wearing_occation,content_category
397,640,。内容：新款限量羊皮Hobo腋下包上身高级感满满简约低调的包型搭配经典logo五金好看到连R...,640,Hobo,羊皮,腋下包,no,no,经典logo五金,容量足够,简约低调|高级感,好看到爱不释手|必须收藏,日常,UGC|好物推荐
270,485,Y.S.L圣罗兰｜奥莱黑色女款皮带|简约大气。内容：#皮带 #腰带 #百搭款腰带 #折扣分享...,485,皮带,no,皮带,no,黑色,简约,百搭,简约|大气,no,日常,购物分享|销售人员
409,655,YSL LE 5 À 7小号核桃色。内容：新色#ysl圣罗兰 #无锡恒隆圣罗兰 #不过时包包...,655,LE 5 A 7,no,腋下包,小号,核桃色,no,no,经典,no,no,UGC|购物分享
253,462,休闲褶皱乐福鞋YSL。内容：#圣罗兰#小众时髦鞋子 #手工皮鞋 #懒人鞋一脚蹬款#YSL#昆...,462,其他鞋履,no,乐福鞋,no,no,no,一脚蹬,休闲|商务休闲|小众时髦,no,商务|休闲,UGC|OOTD
614,891,ysl方盒子✈️。内容：11300#ysl #saintlaurent #盒子包 #折扣分享...,891,其他成衣,no,方包,no,no,no,no,no,no,no,购物分享|UGC


In [11]:
# **switch output file version number here

content_tagging_with_content.to_csv("/root/ysl_project_clean/deepseek_version_test/ds-r1-0528_cleaning_v2.csv")

# Quality assessment

In [12]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

content_tagging_with_content = content_tagging_with_content.replace("no", np.nan)

# 要排除的列
exclude_cols = ['id', 'title_content_hashtag_cleaned']

# 筛选要分析的列
cols_to_check = [c for c in content_tagging_with_content.columns if c not in exclude_cols]

# === 1️⃣ 空值比例与唯一值统计 ===
summary = []
for col in cols_to_check:
    null_ratio = content_tagging_with_content[col].isna().mean()
    unique_count = content_tagging_with_content[col].nunique(dropna=True)
    summary.append({
        'column': col,
        'null_ratio(%)': round(null_ratio * 100, 2),
        'unique_count': unique_count
    })

summary_df = pd.DataFrame(summary)
print("=== 🧾 Missing Value & Unique Summary ===")
print(summary_df.sort_values(by='null_ratio(%)', ascending=False))

=== 🧾 Missing Value & Unique Summary ===
                column  null_ratio(%)  unique_count
4             bag_size          77.36            36
2             material          75.53            77
5                color          67.09            62
6       visual_element          66.39           159
7        functionality          60.62           183
10    wearing_occation          53.02           133
8                style          37.69           271
9   emotion_expression          27.85           320
3                shape          26.16           241
11    content_category           3.94           104
1         product_name           2.39            79
0              level_0           0.00           711
